# Ch4. Memory - 과거의 대화를 장 단기 기억하기
- 1. 언어모델에서 대화란 무엇인가
- 2. 문맥에 맞는 답변을 할 수 있는 챗봇 만들기
- 3. 히스토리를 데이터베이스에 저장하고 영속화하기
- 4. 여러개의 대화를 가질 수 있는 챗봇 만들기
- 5. 매우 긴 대화기록에 대응

## 1.언어모델에서 대화란 무엇인가
> 언어모델과의 상호작용을 저장/복원하여 기억을 생성

- 일회성으로 불러온 경우

In [5]:
from langchain.chat_models import ChatOpenAI  
from langchain.schema import HumanMessage  
from dotenv import load_dotenv
import openai
import warnings
import os
warnings.filterwarnings('ignore')
load_dotenv()

chat = ChatOpenAI(  
    model="gpt-3.5-turbo",  
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature = 0.5
)

result = chat( 
    [
        HumanMessage(content="계란찜을 만드는 재료를 알려주세요"),
    ]
)
print(result.content)

계란찜을 만드는데 필요한 재료는 다음과 같습니다:

- 계란
- 물
- 소금
- 설탕
- 간장
- 다진 마늘
- 다진 파
- 다진 양파
- 다진 고추
- 참기름
- 후추

이 외에도 취향에 따라 다양한 재료를 추가할 수 있습니다. 계란찜을 더 맛있게 만들기 위해 고기나 채소를 넣어도 좋습니다.


- 대화를 이어가며 번역하려면 소스코드를 수정하고 다시 언어모델을 호출해야함

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    HumanMessage,
    AIMessage
)

chat = ChatOpenAI(  
    model="gpt-3.5-turbo",  
)

result = chat([
    HumanMessage(content="계란찜을 만드는 재료를 알려주세요"),
    AIMessage( #← 이 언어모델에 AIMessage로 응답 추가
        content="""계란찜을 만드는데 필요한 재료는 다음과 같습니다:

- 계란
- 물
- 소금
- 설탕
- 간장
- 다진 마늘
- 다진 파
- 다진 양파
- 다진 고추
- 참기름
- 후추

이 외에도 취향에 따라 다양한 재료를 추가할 수 있습니다. 계란찜을 더 맛있게 만들기 위해 고기나 채소를 넣어도 좋습니다."""),
    HumanMessage(content="위의 답변을 영어로 번역하세요")#← 메시지를 추가해 번역시킴
])
print(result.content)

Here are the ingredients needed to make steamed egg:

- Eggs
- Water
- Salt
- Sugar
- Soy sauce
- Minced garlic
- Minced green onions
- Minced onions
- Minced chili peppers
- Sesame oil
- Pepper

In addition, you can add various ingredients according to your preference. You can also add meat or vegetables to make the steamed egg more delicious.


> 이렇게 매번 수기로 하지 않아도 대화 기록을 저장/불러올수 있는 Memory모듈

## 2. 문맥에 맞는 답변을 할 수 있는 챗봇 만들기
> 대화기록을 저장하고 불러오는 기능을 만들어보고, 문맥에 맞는 대답을 하는 챗봇서비스를 만들 예정
- ConversationBufferMemory로 메모리 기능을 구현

In [8]:
'''
가장 기본적인 속성인 ConversationBufferMemory를 알아보자
'''

from langchain.memory import ConversationBufferMemory 
memory = ConversationBufferMemory( #← 메모리 초기화
    return_messages=True, # chat models에서 memory모듈을 사용할수 있으려면 True로 설정
) 
memory.save_context( # save_context로 메모리에 메시지를 추가
    {
        "input": "안녕하세요!"
    },
    {
        "output": "안녕하세요! 잘 지내고 계신가요? 궁금한 점이 있으면 알려 주세요. 어떻게 도와드릴까요?"
    }
)
memory.save_context( #← 메모리에 메시지를 추가
    {
        "input": "오늘 날씨가 좋네요"
    },
    {
        "output": "저는 AI이기 때문에 실제 날씨를 느낄 수는 없지만, 날씨가 좋은 날은 외출이나 활동을 즐기기에 좋은 날입니다!"
    }
)

print(
    memory.load_memory_variables({}) #← 메모리 내용을 확인
)

{'history': [HumanMessage(content='안녕하세요!'), AIMessage(content='안녕하세요! 잘 지내고 계신가요? 궁금한 점이 있으면 알려 주세요. 어떻게 도와드릴까요?'), HumanMessage(content='오늘 날씨가 좋네요'), AIMessage(content='저는 AI이기 때문에 실제 날씨를 느낄 수는 없지만, 날씨가 좋은 날은 외출이나 활동을 즐기기에 좋은 날입니다!')]}


- 히스토리로 HumanMessage, AIMessage 기록

In [ ]:
'''
전과 다르게 메모리를 활용하여 문맥을 기억하는 챗봇을 만들자
chainlit run chat_memory_1.py --port 8001
'''
import chainlit as cl
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory  #← ConversationBufferMemory 가져오기
from langchain.schema import HumanMessage
from dotenv import load_dotenv
import openai
import warnings
import os
warnings.filterwarnings('ignore')
load_dotenv()

chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    api_key=os.getenv("OPENAI_API_KEY"),
)

memory = ConversationBufferMemory( #← 메모리 초기화
    return_messages=True
)

@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="저는 대화의 맥락을 고려해 답변할 수 있는 채팅봇입니다. 메시지를 입력하세요.").send()

@cl.on_message
async def on_message(message: str):
    memory_message_result = memory.load_memory_variables({}) #←유저입력을 넘기기 전 메모리 내용부터 로드

    messages = memory_message_result['history'] #← 메모리 내용에서 메시지(human+AI)만 얻음

    messages.append(HumanMessage(content=message)) # 리스트처럼 append로 방금 메시지(human+AI)를 추가

    result = chat( 
        messages #← Chat models에 messages를 넘긴다
    )

    memory.save_context(  #← 메모리에 메시지를 추가
        {
            "input": message,  #← 사용자의 메시지를 input으로 저장
        },
        {
            "output": result.content,  #← AI의 메시지를 output으로 저장
        }
    )
    await cl.Message(content=result.content).send() #← AI의 메시지를 송신

In [44]:
'''
ConversationChain을 활용하면, 코드가 짧아짐
chat_memory_2.py의 내용을 아래 내용으로 변경후
chainlit run chat_memory_2.py --port 8001
'''
import chainlit as cl
from langchain.chains import ConversationChain  #← ConversationChain을 가져오기
from langchain_anthropic import ChatAnthropic
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
import openai
import warnings
import os
warnings.filterwarnings('ignore')
load_dotenv()

chat = ChatAnthropic(
    model="claude-3-haiku-20240307"
)

memory = ConversationBufferMemory( # ConversationBufferMemory로 메모리생성
    return_messages=True
)

chain = ConversationChain( #← ConversationChain으로 memory+llm 파이프라인 생성
    memory=memory,
    llm=chat,
)

@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="저는 대화의 맥락을 고려해 답변할 수 있는 채팅봇입니다. 메시지를 입력하세요.").send()

@cl.on_message
async def on_message(message: str):

    result = chain( #← 위의 chain 가져옴
        message #← 사용자 메시지를 인수로 지정
    )

    await cl.Message(content=result["response"]).send()

## 3. 히스토리를 데이터베이스에 저장하고 영속화
> 대화를 데이터베이스에 저장해 프로그램이 종료되도 기록이 삭제되지 않도록 한다
- 대화기록을 저장할 데이터베이스로 Redis를 활용
    + 레디스는 캐시, 메시징큐, 단기메모리 등으로 사용되는 고속 오픈소스 인메모리저장시스템
    + key-value 쌍으로 저장되며 다양한 유형 지원(목록열,집합,해시,비트맵,하이퍼로그등)
    + 메인 메모리에 데이터를 저장하기에 디스크기반 db보다 빠름
    + 메모리 내 데이터는 휘발성이 있지만, 레디스는 주기적으로 디스크에 데이터를 기록함으로 영속성 제공
    + 확장성과 고가용성을 보장하기 위한 복제/샤딩 기능도 갖추고 있음
[upstash](https://upstash.com/)에서 redis를 이용하자
[Langchain가이드라인](https://python.langchain.com/docs/integrations/memory/upstash_redis_chat_message_history/)

In [38]:
# # for sync client
# from upstash_redis import Redis

# redis = Redis(url="UPSTASH_URL", token="UPSTASH_REDIS_REST_TOKEN")

# for async client
from upstash_redis.asyncio import Redis

redis = Redis(url=os.getenv("UPSTASH_REDIS_REST_URL"), token=os.getenv("UPSTASH_REDIS_REST_TOKEN"))
redis.set("foo", "bar")
value = redis.get("foo")
print(value)

<coroutine object Redis.execute at 0x000001FC39514EB0>


In [40]:
from langchain_community.chat_message_histories import (
    UpstashRedisChatMessageHistory,)

URL = 
TOKEN = 

history = UpstashRedisChatMessageHistory(
    url=os.getenv("UPSTASH_REDIS_REST_URL"), token=os.getenv("UPSTASH_REDIS_REST_TOKEN"), 
    ttl=10, 
    session_id="my-test-session"
)

history.add_user_message("hello llm!")
history.add_ai_message("hello user!")

In [1]:
import os  #← 환경변수를 얻기 위해 os를 가져오기
import chainlit as cl
from langchain.chains import ConversationChain
from langchain_anthropic import ChatAnthropic
from langchain_community.chat_message_histories import UpstashRedisChatMessageHistory
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv

load_dotenv()
message = '계란찜 만드는 방법 알려줘'

chat = ChatAnthropic(
    model="claude-3-haiku-20240307",
    api_key=os.getenv("Anthropic_API_KEY")
)
# 장기기억으로 upstash의 redis 이용
'''
대화내역이 redis에 저장되지 때문에 애플리케이션 종료 후에도 내역이 유지됨
'''
history = UpstashRedisChatMessageHistory(
    url=os.getenv("UPSTASH_REDIS_REST_URL"), token=os.getenv("UPSTASH_REDIS_REST_TOKEN"), 
    session_id="my-test-session"
)

memory = ConversationBufferMemory(
    return_messages=True,
    chat_memory=history,  #← 채팅 기록을 지정
)

chain = ConversationChain(
    memory=memory,
    llm=chat,
)

result = chain(message)
result["response"]

2024-05-06 11:22:24 - Loaded .env file


E:\miniconda\envs\lang310\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


2024-05-06 11:22:29 - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


'네, 계란찜 만드는 방법은 다음과 같습니다:\n\n준비물:\n- 계란 6개\n- 다진 파 2큰술\n- 액젓 1큰술\n- 설탕 1작은술\n- 소금 약간\n- 물 1과 1/2컵\n\n만드는 방법:\n1. 계란을 그릇에 담아 물을 부어 섞어줍니다. 파, 액젓, 설탕, 소금을 넣고 다시 잘 섞어줍니다.\n2. 준비한 그릇을 찜통에 넣고 20분 정도 찌면 됩니다. \n3. 중간에 한 두 번 계란을 저어주면 부드러운 질감의 계란찜을 만들 수 있습니다.\n4. 계란이 완전히 익으면 그릇을 꺼내 식힌 후 담아내면 완성입니다.\n\n계란의 신선도와 개인의 맛 선호도에 따라 약간씩 조절하면 더 맛있는 계란찜을 만들 수 있습니다. 궁금한 점이 더 있다면 물어보세요.'

In [ ]:
'''
기존의 내용을 기억하는 챗봇을 실행하고, 종료했다가 다시 이어서 실행해보세요
chat_memory_3.py의 내용을 아래 내용으로 변경후
chainlit run chat_memory_3.py --port 8002
'''
import os  #← 환경변수를 얻기 위해 os를 가져오기
import chainlit as cl
from langchain.chains import ConversationChain
from langchain_anthropic import ChatAnthropic
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
from langchain_community.chat_message_histories import     UpstashRedisChatMessageHistory
load_dotenv()

chat = ChatAnthropic(
    model="claude-3-haiku-20240307",
    api_key=os.getenv("Anthropic_API_KEY")
)

history = UpstashRedisChatMessageHistory(
    url=os.getenv("UPSTASH_REDIS_REST_URL"),
    token=os.getenv("UPSTASH_REDIS_REST_TOKEN"), 
    session_id="chat_history")

memory = ConversationBufferMemory(
    return_messages=True,
    chat_memory=history,  #← 채팅 기록을 지정
)

chain = ConversationChain(
    memory=memory,
    llm=chat,
)

@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="저는 대화의 맥락을 고려해 답변할 수 있는 채팅봇입니다. 메시지를 입력하세요.").send()

@cl.on_message
async def on_message(message: str):

    result = chain(message)

    await cl.Message(content=result["response"]).send()


> 어플리케이션을 새로 시작해도 history의 session_id 만 같으면 전의 내용을 기억합니다.

## 4. 여러개의 대화기록을 가질 수 있는 챗봇 만들기
> 기존 시스템은 session_id를 변경이 불가능하여, 다른 대화를 할 수 없었다. 이를 할 수 있도록 하여, 이전에 어떤 대화를 했는지 복원하자

In [ ]:
import os
import chainlit as cl
from langchain.chains import ConversationChain
from langchain_anthropic import ChatAnthropic
from langchain.memory import ConversationBufferMemory, RedisChatMessageHistory
from langchain_community.chat_message_histories import     UpstashRedisChatMessageHistory

from langchain.schema import HumanMessage
from dotenv import load_dotenv

load_dotenv()

chat = ChatAnthropic(
    model="claude-3-haiku-20240307",
    api_key=os.getenv("Anthropic_API_KEY"),
)

@cl.on_chat_start
async def on_chat_start():
    thread_id = None
    while not thread_id: #← 스레드 ID가 입력될 때까지 반복
        res = await cl.AskUserMessage(content="저는 대화의 맥락을 고려해 답변할 수 있는 채팅봇입니다. 스레드 ID를 입력하세요.", timeout=600).send() #← AskUserMessage를 사용해 스레드 ID 입력
        if res:
            thread_id = res['content']

    history = UpstashRedisChatMessageHistory(
    url=os.getenv("UPSTASH_REDIS_REST_URL"),
    token=os.getenv("UPSTASH_REDIS_REST_TOKEN"), 
    session_id="my-test-session"
)


    memory = ConversationBufferMemory( #← 새로 채팅이 시작될 때마다 초기화하도록 on_chat_start로 이동
        return_messages=True,
        chat_memory=history,
    )

    chain = ConversationChain( #← 새로 채팅이 시작될 때마다 초기화하도록 on_chat_start로 이동
        memory=memory,
        llm=chat,
    )

    memory_message_result = chain.memory.load_memory_variables({}) #← 메모리 내용 가져오기

    messages = memory_message_result['history']

    for message in messages:
        if isinstance(message, HumanMessage): #← 사용자가 보낸 메시지인지 판단
            await cl.Message( #← 사용자 메시지이면 authorUser를 지정해 송신
                author="User",
                content=f"{message.content}",
            ).send()
        else:
            await cl.Message( #← AI의 메시지이면 ChatBot을 지정해 송신
                author="ChatBot",
                content=f"{message.content}",
            ).send()
    cl.user_session.set("chain", chain) #← 기록을 세션에 저장

@cl.on_message
async def on_message(message: str):
    chain = cl.user_session.get("chain") #← 세션에서 기록을 가져오기

    result = chain(message)

    await cl.Message(content=result["response"]).send()


## 5. 매우 긴 대화 기록에 대응
> 대화 기록을 영속화하고 과거 대화 기록도 가져올 수 있게 되었다. 하지만 대화가 매우 길어지면, LLM 입력에 한계가 있기에 에러가 발생한다. 이러한 에러를 해결해보자

In [8]:
'''
에러가 발생하는 사례 
'''
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage
load_dotenv()
chain = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"))
result = chat([
    HumanMessage(content="계란찜 만드는 방법을 알려줘"),
    AIMessage(content="{ChatGPT의 답변인 계란찜 만드는 방법}"),
    HumanMessage(content="만두 빚는 방법을 알려줘")    ,
    AIMessage(content="{ChatGPT의 답변인 만두 빚는 방법}"),
    HumanMessage(content="볶음밥 만드는 방법을 알려줘")
])   
print(result.content)

2024-05-06 21:05:16 - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
{ChatGPT의 답변인 볶음밥 만드는 방법}


### 오래된 대화 삭제하기
- 최근 k개의 대화를 남기고 오래된 대화를 삭제
- ConversationBufferWindowMemory

In [ ]:
'''
오래된 대화를 단순 삭제, 예>최근 3개만 기억
chainlit run custom_memory_1.py --port 8002
'''

import chainlit as cl
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory  # ConversationBufferWindowMemory 가져오기
load_dotenv()
chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    api_key=os.getenv("OPENAI_API_KEY")
)

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=3 # 3번 주고받은 메시지를 기억
)

chain = ConversationChain(
    memory=memory,
    llm=chat,
)

@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="저는 대화의 맥락을 고려해 답변할 수 있는 채팅봇입니다. 메시지를 입력하세요.").send()

@cl.on_message
async def on_message(message: str):
    messages = chain.memory.load_memory_variables({})["history"] # 저장된 메시지 가져오기

    print(f"저장된 메시지 개수: {len(messages)}" # 저장된 메시지 개수를 표시
          )

    for saved_message in messages: # 저장된 메시지를 1개씩 불러옴
        print(saved_message.content # 저장된 메시지를 표시
              )

    result = chain(message)

    await cl.Message(content=result["response"]).send()


### 지난 대화를 요약하여 토큰수 제한에 대응
- 대화별 내용 요약
- ConversationSummaryMemory

In [ ]:
'''
오래된 대화를 단순 삭제, 예>최근 3개만 기억
chainlit run custom_memory_2.py --port 8002
'''

import chainlit as cl
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory  # ConversationBufferWindowMemory 가져오기
load_dotenv()
chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    api_key=os.getenv("OPENAI_API_KEY")
)

memory = ConversationSummaryMemory(
    llm=chat,     # 요약 모델 지정
    return_messages=True
)

chain = ConversationChain(
    memory=memory,
    llm=chat,
)

@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="저는 대화의 맥락을 고려해 답변할 수 있는 채팅봇입니다. 메시지를 입력하세요.").send()

@cl.on_message
async def on_message(message: str):
    messages = chain.memory.load_memory_variables({})["history"] # 저장된 메시지 가져오기

    print(f"저장된 메시지 개수: {len(messages)}") # 저장된 메시지 개수를 표시
         

    for saved_message in messages: # 저장된 메시지를 1개씩 불러옴
        print(saved_message.content) # 저장된 메시지를 표시
              
    result = chain(message)
    await cl.Message(content=result["response"]).send()